# ReRAM Crossbar: Ideal vs Non-Ideal Modeling

This notebook compares a mathematically perfect crossbar (Ohm's Law) with a physical ReRAM model that incorporates:
1. **Device Variability**: Random fluctuations in conductance.
2. **Interconnect Noise**: Signal-dependent noise during readout.
3. **Cycle-to-Cycle Variability**: Conductance drift.
4. **IR Drop (Wire Resistance)**: Voltage degradation along the bitlines/wordlines.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Ensure project root is available
sys.path.insert(0, str(Path().resolve().parent))

from src.crossbar import IdealCrossbar, NonIdealCrossbar
from src.utils.visualization import plot_heatmap

np.random.seed(42)
ROWS, COLS = 64, 64

# 1. Setup Models
G = np.random.rand(ROWS, COLS).astype(np.float32) * 0.1
V = np.random.rand(ROWS).astype(np.float32) * 0.5

ideal = IdealCrossbar(ROWS, COLS)
ideal.set_conductance(G)

non_ideal = NonIdealCrossbar(
    ROWS, COLS, 
    noise_std=0.01, 
    variability_std=0.02, 
    ir_drop_row_r=0.05,
    seed=42
)
non_ideal.set_conductance(G)

# 2. Run Inference
I_ideal = ideal.run(V)
I_non = non_ideal.run(V)

# 3. Analysis
mse = np.mean((I_ideal - I_non)**2)
snr = 10 * np.log10(np.mean(I_ideal**2) / (mse + 1e-9))

print(f"Results for {ROWS}x{COLS} Crossbar:")
print(f"  Mean Squared Error (MSE): {mse:.6e}")
print(f"  Signal-to-Noise Ratio (SNR): {snr:.2f} dB")

# 4. Plotting
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(I_ideal, label='Ideal', alpha=0.7)
plt.plot(I_non, label='Non-Ideal', alpha=0.7)
plt.title("Output Current (Bitlines)")
plt.xlabel("Column Index")
plt.ylabel("Current (I)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.subplot(1, 2, 2)
plt.hist(I_ideal - I_non, bins=30, color='crimson', alpha=0.7)
plt.title("Error Distribution (I_ideal - I_non)")
plt.xlabel("Error Value")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()